# Determine number of logged-in editors using each skin on English Wikipedia

[TICKET:T316884](https://phabricator.wikimedia.org/T316884)

In [1]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
library(tidyverse); 
library(lubridate); 
library(scales);
library(magrittr); 
library(dplyr);
})

In [31]:
library(glue)

In [2]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [5]:
query_editor <- "
 SELECT
        event_user_text AS user_name,
        event_user_id AS user_id,
        count(revision_id) AS content_edits
    FROM wmf.mediawiki_history 
    WHERE
        wiki_db ='enwiki' AND snapshot='2022-08' AND
        event_entity = 'revision' AND
        event_type = 'create' AND
        --logged-in users
        event_user_is_anonymous = false AND
        --exclude bots
        size(event_user_is_bot_by) = 0 AND
        not array_contains(event_user_groups, 'bot') AND
        --content edits only
        page_namespace_is_content = true AND
        substr(event_timestamp,1,10) BETWEEN '2021-09-01' AND '2022-08-31' 
    GROUP BY event_user_text, event_user_id  
"

In [6]:
df_active_editor <- wmfdata::query_hive(query_editor)

Don't forget to authenticate with Kerberos using kinit



In [36]:
users_list <- toString(df_active_editor$user_id)

In [29]:
query_nondefault_skin <- " 
SELECT up_user AS user_id,
  up_value AS skin
FROM user_properties
WHERE up_user in (${users_list})
AND up_property = 'skin'
"

In [37]:
query <- glue(query_nondefault_skin, .open = "${", .close = "}")

In [39]:
df_nondefault_skin <- wmfdata::mysql_read(query, 'enwiki')

Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 0 imported as numeric”
Fetched 44768 rows and 2 columns.



In [ ]:
df_active_editor 

In [41]:
df <- 
merge(df_active_editor ,df_nondefault_skin ,  by.x=c("user_id"),by.y=c("user_id"),  all.x = TRUE)


In [47]:
df_summary <- df %>%
group_by(skin) %>%
summarize( counts=n(),  .groups='drop') 


In [48]:
df_summary

skin,counts
<chr>,<int>
,15443
0,223
1,5
2,13
amethyst,1
chick,47
cologneblue,485
minerva,1203
modern,1273


How to interpret skin value in `user_properties` table 

doc ref: https://phabricator.wikimedia.org/T180860,   

code ref: https://github.com/wikimedia/mediawiki/blob/master/includes/skins/Skin.php

In [55]:
#skin map
df_summary <- df_summary %>%
mutate(
  skin_name = case_when (
      #doc ref: https://phabricator.wikimedia.org/T180860, 
      #code ref: https://github.com/wikimedia/mediawiki/blob/master/includes/skins/Skin.php
      skin=='vector-2022' ~ 'vector-2022',
      skin=='vector' ~ 'vector',
      skin=='NA' ~ 'vector',
      skin=='' ~ 'vector',
      skin=='0' ~ 'vector',
      skin=='1' ~ 'vector',
      skin=='simple' ~ 'vector',
      skin=='nostalgia' ~ 'vector',
      skin=='chick' ~ 'vector',
      skin=='standard' ~ 'vector',
      skin=='classic' ~ 'vector',
      skin=='cologneblue' ~ 'cologneblue',
      skin=='2' ~ 'cologneblue',
      skin=='myskin' ~ 'monobook',
      skin=='monobook' ~ 'monobook',
      skin=='minerva' ~ 'minerva',
      skin=='modern' ~ 'modern',
      skin=='timeless' ~ 'timeless',
      skin=='amethyst' ~ 'amethyst',
      TRUE ~ 'vector'
        )
   )



In [56]:
df_summary

skin,counts,skin_name
<chr>,<int>,<chr>
,15443,vector
0,223,vector
1,5,vector
2,13,cologneblue
amethyst,1,amethyst
chick,47,vector
cologneblue,485,cologneblue
minerva,1203,minerva
modern,1273,modern


In [58]:
df_editors_per_skin <- df_summary %>%
 group_by(skin_name) %>%
 summarize(num_editors=sum(counts))

In [61]:
total_editors <- sum(df_editors_per_skin$num_editors)

In [62]:
total_editors

[1] 628101

In [65]:
df_editors_per_skin <- df_editors_per_skin %>%
  mutate(
  editors_pct = round(num_editors*100/total_editors,2)
  )

In [66]:
df_editors_per_skin

skin_name,num_editors,editors_pct
<chr>,<int>,<dbl>
amethyst,1,0.00
cologneblue,498,0.08
minerva,1203,0.19
modern,1273,0.20
monobook,13494,2.15
timeless,2435,0.39
vector,599896,95.51
vector-2022,9301,1.48
